In [ ]:
import praw
import pickle

In [ ]:
#Unify into one dictionary
#These are charity mission statements that can be matched against posts/comments
with open('orgID_missionDict.pickle', 'rb') as handle1:
    orgID_missionDict = pickle.load(handle1)
    
with open('orgID_websiteDict.pickle', 'rb') as handle2:
    orgID_websiteDict = pickle.load(handle2)
    
with open('orgID_NameDict.pickle', 'rb') as handle3:
    orgID_NameDict = pickle.load(handle3)

In [ ]:
#Import nltk components to use for matching (TF) instance
#Can substitute other models here
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

#Get stem funciton
def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

#Get stems, remove punctuation, transform lower case
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1] #positions in the matrix for the similarity


In [ ]:
#Register Credentials
reddit = praw.Reddit(client_id="******",
                     client_secret="******",
                     user_agent="******",
                     username = '******',
                     password = '******')


In [ ]:
from operator import itemgetter


subreddit = reddit.subreddit("test")

#For basic usage to attain submission titles/selftext 
#and highest level comments
#for submission in subreddit.hot(limit=10):
value_list_nltk = []
#Stream usage otherwise same as basic usage
for submission in subreddit.stream.submissions():

    submission_title = submission.title
    submission_selftext = submission.selftext
    
    #submission_flairID = submission.link_flair_template_id
    #submission_flairText = submission.link_flair_text
    
    if "!charitybot" in submission_title or "!charitybot" in submission_selftext:
        print("word charity found")
        #submission.reply(reply_text)
        
        for key, value in orgID_missionDict.items():
            #value_list.append(charity_text.similarity(nlp(value)))
            value_list_nltk.append([cosine_sim(submission_selftext, value), key])
    
        value_list_sorted = sorted(value_list_nltk, key=itemgetter(0))
        
        
        first_Recom_key = value_list_sorted[-1][1]
        second_Recom_key = value_list_sorted[-2][1]
        third_Recom_key = value_list_sorted[-3][1]
        
        interest_text = "The following organization(s) may be of interest to you: <br>"
        first_Recom_Mission = orgID_missionDict[first_Recom_key]
        first_Recom_URL = orgID_websiteDict[first_Recom_key]
        first_Recom_Name = orgID_NameDict[first_Recom_key]
        
        submission.reply(interest_text + first_Recom_Name + " : "  + first_Recom_URL + "\n" + " Mission Statement: "\
                        + first_Recom_Mission)
        
        
        
        print("recommendation submitted")